# Bulk processor and improved FlySight file and jump validation

Uses `./data` as a data lake.

In [ ]:
%%capture
!x=$(pip list | awk '/haversine/') ; [[ -z "$x" ]] && make local

In [5]:
from pathlib import Path

import warnings

import bokeh.models as bm
import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [28]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import getFlySightDataFromCSVFileName
from ssscoring.calc import isValidMaximumAltitude
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import processJump
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.datatypes import JumpStatus
from ssscoring.constants import FT_IN_M
from ssscoring.flysight import getAllSpeedJumpFilesFrom
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

---
## Set DZ altitude MSL

Set the value in ft.  Wingsuit World <a href='https://wingsuit.world/dropzones/' target='_blank'>List of Dropzones</a> is a good resource.

In [29]:

dropZones = {
    'Drop zone': [
        'Aerodrom Tanay',
        'Aerograd Kolomna',
        'Bay Area Skydiving',
        'Drop Zone Thailand',
        'DZ Krutitcy',
        'Fehrbellin',
        'Lodi Parachute Center',
        'Mile High',
        'Neustadt-Glewe',
        'Paracaidismo Celaya',
        'Paraclete XP',
        'Saarlouis-Düren',
        'SkyDance SkyDiving',
        'Skydive Algarve',
        'Skydive Arizona',
        'Skydive Buzz',
        'Skydive Chicago',
        'Skydive Fano',
        'Skydive Netheravon',
        'Skydive Perris',
        'Skydive Puebla',
        'Skydive Saulgau',
        'Skydive Teuge',
        'Thai Sky Adventures',
    ],
    'Alt (ft)': [
        699.0,
        472.0,
        23.0,
        15.0,
        377.0,
        138.0,
        59.0,
        5500.0,
        115.0,
        5734.0,
        304.0,
        1119.0,
        100.0,
        6.0,
        1509.0,
        840.0,
        616.0,
        52.0,
        454.0,
        1414.0,
        5744.0,
        1903.0,
        15.0,
        21.0,
    ],
}

pd.DataFrame(dropZones, columns=[ 'Drop zone', 'Alt (ft)', ])

,Drop zone,Alt (ft)
0,Aerodrom Tanay,699.0
1,Aerograd Kolomna,472.0
2,Bay Area Skydiving,23.0
3,Drop Zone Thailand,15.0
4,DZ Krutitcy,377.0
5,Fehrbellin,138.0
6,Lodi Parachute Center,59.0
7,Mile High,5500.0
8,Neustadt-Glewe,115.0
9,Paracaidismo Celaya,5734.0


In [30]:
dropZoneAltMSL = 15.0
ignoreBaseline = True

In [31]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">  15.00 ft</span> (   4.57 m)<h1>')

In [32]:
jumpFiles = getAllSpeedJumpFilesFrom(Path('./data'))

---
## Process jump file


In [34]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpFilesList = list(jumpFiles.keys())
if (len(jumpFilesList) > 1):
    filePath = jumpFilesList[1]
    rawData, tag = getFlySightDataFromCSVFileName(filePath)
    data = convertFlySight2SSScoring(rawData, altitudeDZMeters=dropZoneAltMSLMeters)
    jumpResult = processJump(data)
    if jumpResult.status == JumpStatus.OK:
        display(jumpResult.table)
        display(jumpResult.window)

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
8360,5.0,171.972,131.877100,52.52,204.90,13307.505319,216.716262
8385,10.0,310.464,94.890189,73.00,365.79,12183.476993,324.641407
8410,15.0,409.140,65.271246,80.94,468.14,10511.915954,414.313740
8435,20.0,459.000,6.779575,89.15,516.96,8513.098397,459.050066
8460,25.0,462.240,27.270636,86.62,532.16,6398.153642,463.043740


PerformanceWindow(start=np.float64(4185.560944403804), end=np.float64(1929.5609444038037), validationStart=np.float64(2935.5609444038037))

---
## Results

In [35]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpResults = processAllJumpFiles(jumpFiles=jumpFiles, altitudeDZMeters=dropZoneAltMSLMeters)
aggregate = aggregateResults(jumpResults)
aggregate

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 10-35-35:v1,467.91,175.032,316.944,414.252,458.64,464.832,24.8,469.404
data 12-01-20:v1,465.27,171.972,310.464,409.140,459.00,462.240,25.0,465.984


### Rounded results for training log

In [36]:
roundedResults = roundedAggregateResults(aggregate)
roundedResults

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 10-35-35:v1,468,175,317,414,459,465,24.8,469
data 12-01-20:v1,465,172,310,409,459,462,25.0,466


## All jumps

In [25]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)

In [26]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    if ignoreBaseline and 'baseline' in resultRef:
        continue
    jumpResult = jumpResults[resultRef]
    if jumpResult.status == JumpStatus.OK:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.01f km/h</span></h1>' % ('green', resultRef, 'VALID', jumpResult.score)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - %s</span></h1>' % ('red', resultRef, 'INVALID', jumpResult.status)

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.status == JumpStatus.OK:
        belowMaxAltitude = isValidMaximumAltitude(jumpResult.data.altitudeAGL.max())
        badJumpLegend = None
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            badJumpLegend = '<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL was lower than the minimum scoring altitude according to IPC and USPA.</h3>'
        if not belowMaxAltitude:
            badJumpLegend = '<h3><span style="color: red"><span style="font-weight: bold">RE-JUMP:</span> exit altitude AGL exceeds the maximum altitude according to IPC and USPA.</h3>'
            validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - %s</span></h1>' % ('red', resultRef, 'INVALID', JumpStatus.ALTITUDE_EXCEEDS_MAXIMUM)
        display(widgets.HTML(validJumpStatus))            
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3?'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if badJumpLegend:
            display(widgets.HTML(badJumpLegend))
            # TODO: Fix this logic, it's bass ackwards.
            # if not belowMaxAltitude:
            #     continue
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips=[('Y-val', '@y{0.00}',),])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(
            individualPlot,
            jumpResult,
            lineColor=SPEED_COLORS[0])
        graphJumpResult(
            allJumpsPlot,
            jumpResult,
            lineColor=SPEED_COLORS[mixColor],
            legend='%s - %.2f' % (resultRef, jumpResult.score),
            showIt=False)

HTML(value='<hr><h1><span style="color: green">data 10-35-35:v1 jump - VALID - score = 467.9 km/h</span></h1>'…

HTML(value='<h3>Max speed = 469; exit at 4198 m (13775 ft), end scoring window at 1942 m (6374 ft)</h3?')

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,175,118,56.09,192.51,13347,211
,10.0,317,93,73.62,338.90,12194,330
,15.0,414,52,82.79,437.17,10493,418
,20.0,459,13,88.38,474.43,8465,459
,25.0,465,23,87.14,464.27,6424,465


---
## All skydives

In [ ]:
sumResults = totalResultsFrom(aggregate)
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)